In [55]:
baked_path = "G:\\three-d-map230818\\public\\merf-qizhong\\data\\block_1\\baked\\baked"
out_path = "G:\\three-d-map230818\\public\\merf-qizhong\\data\\block_1\\baked\\baked_cut"

In [75]:
import numpy as np
from PIL import Image
import shutil
import json 
plane_res = 4096
factor = 8
block_id = 0
if block_id == 0:
    indicator_buond_xz = [[True,True],[True,False]] #x_min,y_min,x_max,y_max
if block_id == 1:
    indicator_buond_xz = [[True,True],[True,False]] #x_min,y_min,x_max,y_max
if block_id == 2:
    indicator_buond_xz = [[True,True],[True,False]] #x_min,y_min,x_max,y_max
if block_id == 3:
    indicator_buond_xz = [[True,True],[True,False]] #x_min,y_min,x_max,y_max
xz_bound = [plane_res//4 - plane_res//40,plane_res*3//4 + plane_res//40]
print(xz_bound)
y_bound = [1480,2360]
shutil.copytree(baked_path,out_path)



[922, 3174]


FileExistsError: [WinError 183] 当文件已存在时，无法创建该文件。: 'G:\\three-d-map230818\\public\\merf-qizhong\\data\\block_1\\baked\\baked_cut'

In [79]:
plane = ["\\plane_features_","\\plane_rgb_and_density_"]
for i in range(3):
    for name in plane:
        image_path = baked_path  + name + str(i) + ".png"
        image = Image.open(image_path)
        image_data = np.array(image)
        non_zero_coords = np.argwhere(np.all(image_data != 0, axis=2))
        x_min, y_min = non_zero_coords.min(axis=0)
        x_max, y_max = non_zero_coords.max(axis=0)
        print(x_min, y_min, x_max, y_max)
        if name=="\\plane_features_":
            if i == 0:
                plane_bound_min = [x_min, y_min] 
                plane_bound_max = [x_max, y_max]
            if i == 1:
                plane_bound_min.append(y_min)
                plane_bound_max.append(y_max)
        # print(plane_bound_min)
        # print(plane_bound_max)
plane_bound = np.array([plane_bound_min, plane_bound_max])
print(plane_bound)
real_y_min = y_bound[0]
real_y_max = y_bound[1]
if indicator_buond_xz[0][0]:
    real_x_min = max(plane_bound[0][2],xz_bound[0])
else:
    real_x_min = plane_bound[0][0]
if indicator_buond_xz[0][1]:
    real_z_min = max(plane_bound[0][0],xz_bound[0])
else:
    real_z_min = plane_bound[0][2]
if indicator_buond_xz[1][0]:
    real_x_max = min(plane_bound[1][2],xz_bound[1])
else:
    real_x_max = plane_bound[1][0]
if indicator_buond_xz[1][1]:
    real_z_max = min(plane_bound[1][0],xz_bound[1])
else:
    real_z_max = plane_bound[1][2]
real_bound = np.array([[real_x_min,real_y_min,real_z_min],[real_x_max,real_y_max,real_z_max]])
real_bound_float = real_bound.astype(np.float32)/4096 * 4 - 2.0
print(real_bound)
for i in range(3):
    for name in plane:
        image_path = baked_path  + name + str(i) + ".png"
        image = Image.open(image_path)
        image_data = np.array(image)
        print(image_data.shape)
        if i == 0:
            cropped_image = image.crop((real_bound[0][1] - 1,  real_bound[0][0] - 1, real_bound[1][1] + 1, real_bound[1][0] + 1))
        if i == 1:
            cropped_image = image.crop((real_bound[0][2] - 1,  real_bound[0][0] - 1, real_bound[1][2] + 1, real_bound[1][0] + 1))
        if i == 2:
            cropped_image = image.crop((real_bound[0][2] - 1,  real_bound[0][1] - 1, real_bound[1][2] + 1, real_bound[1][1] + 1))
        cropped_image_path = out_path + name + str(i) + ".png"
        cropped_image.save(cropped_image_path)
        
real_bound_float_f = np.zeros((2,3))
for i in range(2):
    for j in range(3):
        if real_bound_float[i][j] < -1.0 or real_bound_float[i][j] > 1.0:
            real_bound_float_f[i][j] = 1/(2.0-np.abs(real_bound_float[i][j])) * np.sign(real_bound_float[i][j])
        else:
            real_bound_float_f[i][j] = real_bound_float[i][j]
            
real_bound_list = real_bound.tolist()
real_bound_float_list = real_bound_float_f.tolist()
real_bound_contract = real_bound_float.tolist()

# Define the data to be saved in JSON format
data_to_save = {
    "real_bound": real_bound_list,
    "real_bound_float": real_bound_float_list,
    "real_bound_contract": real_bound_contract,
}
json_file_path = "G:\\three-d-map230818\\public\\merf-qizhong\\data\\block_1\\baked\\baked_cut" + "\\bound_params.json" # Change this to your desired file path


# Write the data to a JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(data_to_save, json_file, indent=2)
    
# print(real_bound) 
# print(real_bound_float)

538 1490 3084 2636
538 1490 3084 2636
538 1011 3084 3439
538 1011 3084 3439
1490 1011 2636 3439
1490 1011 2636 3439
[[ 538 1490 1011]
 [3084 2636 3439]]
[[1011 1480  922]
 [3174 2360 3439]]
(4096, 4096, 4)
(4096, 4096, 4)
(4096, 4096, 4)
(4096, 4096, 4)
(4096, 4096, 4)
(4096, 4096, 4)


In [78]:
real_bound_float_f = np.zeros((2,3))
for i in range(2):
    for j in range(3):
        if real_bound_float[i][j] < -1.0 or real_bound_float[i][j] > 1.0:
            real_bound_float_f[i][j] = 1/(2.0-np.abs(real_bound_float[i][j])) * np.sign(real_bound_float[i][j])
        else:
            real_bound_float_f[i][j] = real_bound_float[i][j]
print(real_bound_float_f)
print(real_bound_float)


[[-1.01285856 -0.5546875  -1.11062907]
 [ 1.11062907  0.3046875   1.5585997 ]]
[[-1.0126953 -0.5546875 -1.0996094]
 [ 1.0996094  0.3046875  1.3583984]]


In [20]:
occ = ["\\occupancy_grid_8.png","\\occupancy_grid_16.png","\\occupancy_grid_32.png","\\occupancy_grid_64.png","\\occupancy_grid_128.png"]
occ_idx = [512,256,128,64,32]
for i in range(5):
    image_path = baked_path  + occ[i]
    image = Image.open(image_path)
    image_data = np.array(image)
    image_data = image_data.reshape(occ_idx[i],occ_idx[i],occ_idx[i],4)
    non_zero_coords = np.argwhere(np.all(image_data != 0, axis=3))
    # print(non_zero_coords.min(axis=0))
    # print(non_zero_coords.max(axis=0))
    # print(image_data.shape)
    if  occ[i] == "\\occupancy_grid_8.png":
        occ_bound = np.array([non_zero_coords.min(axis=0),non_zero_coords.max(axis=0)])
    print(occ_bound)

C:\Users\Kevin\AppData\Roaming\Python\Python38\site-packages\PIL\Image.py:3182: DecompressionBombWarning: Image size (134217728 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


[[ 68 187 127]
 [384 328 428]]
[[ 68 187 127]
 [384 328 428]]
[[ 68 187 127]
 [384 328 428]]
[[ 68 187 127]
 [384 328 428]]
[[ 68 187 127]
 [384 328 428]]


In [49]:
image_path = baked_path  + "\\sparse_grid_block_indices.png"
image = Image.open(image_path)
image_data = np.array(image)
print(image_data.shape)

(4096, 64, 3)
